##### Example 1

In [ ]:
import torch

In [ ]:
world_size = 16

In [ ]:
tensor_model_parallel_size = 2

In [ ]:
pipeline_model_parallel_size = 4

In context of Megatron-LM

In [ ]:
world_size, tensor_model_parallel_size, pipeline_model_parallel_size

(16, 2, 4)

Calculate the number of GPUs requires to parallelize a model. Explain the calculation

**Answer**

In pipeline parallelism, a model is split into `pipeline_model_parallel_size`.

Because Megatron-LM incorporates both tensor parallelism and pipeline parallelism, so each stage has `tensor_model_parallel_size` GPUs to parallelize the tensor operations in that stage. So, the total number of GPUs required to parallelize a model would be:

In [ ]:
num_gpus_for_each_model = tensor_model_parallel_size * pipeline_model_parallel_size

In [ ]:
num_gpus_for_each_model

8

In [ ]:
type(num_gpus_for_each_model)

int

Calculate the number of replicates model in data parallelism. Explain the calculation

**Answer**

To calculate the number of model replicates in data parallelism, we divide the total number of GPUs (`world_size`) by the number of GPUs used for each model (`num_gpus_for_each_model`):

In [ ]:
data_parallel_size = world_size // num_gpus_for_each_model

In [ ]:
data_parallel_size

2

##### Example 2

In [ ]:
world_size = 16
tensor_model_paralell_size = 2
num_tensor_model_parallel_groups = world_size // tensor_model_paralell_size

In [ ]:
list_ranks = []

In [ ]:
def set_current_tensor_parallel_group(group):
    pass

In [ ]:
import torch

In [ ]:
tensor_model_paralell_size, num_tensor_model_parallel_groups

(2, 8)

In [ ]:
list_ranks

[]

In Megatron-LM, write a script that can be distributed to multiple devices to create tensor model parallel groups.

**Hint**: For each GPU, you should specify the group it belongs to by calling the `set_current_tensor_parallel_group(group)` function.

In [ ]:
rank = torch.distributed.get_rank()

AttributeError: module 'torch.distributed' has no attribute 'get_rank'

In [ ]:
for i in range(num_tensor_model_parallel_groups):
    ranks = range(
        i * tensor_model_paralell_size,
        (i+1) * tensor_model_paralell_size
    )
    
    list_ranks.append(ranks) # can be ignore
    
    if rank in ranks:
        group = torch.distributed.new_group(ranks)
        set_current_tensor_parallel_group(group)

NameError: name 'rank' is not defined

In [ ]:
[print(f"Group {i}: {list(x)}") for i, x in enumerate(list_ranks)]

Group 0: [0, 1]


[None]

##### Example 3

In [ ]:
world_size = 16

In [ ]:
pipeline_model_paralell_size = 4

In [ ]:
num_pipeline_model_parallel_groups = world_size // pipeline_model_paralell_size

In [ ]:
num_pipeline_model_parallel_groups

4

In [ ]:
list_ranks = []

In [ ]:
for i in range(num_pipeline_model_parallel_groups):
    ranks = range(i, world_size, num_tensor_model_parallel_groups)
    
    # list_ranks.append(ranks)
    # print(list(ranks))
    print(list(ranks))

[0, 8]
[1, 9]
[2, 10]
[3, 11]


In [ ]:
print(list(list_ranks))

[range(0, 16, 8), range(1, 16, 8), range(2, 16, 8), range(3, 16, 8)]


In [ ]:
[print(f"Group {i}: {list(x)}") for i, x in enumerate(list_ranks)]

Group 0: [0, 8]
Group 1: [1, 9]
Group 2: [2, 10]
Group 3: [3, 11]


[None, None, None, None]

##### Example 4

In [ ]:
import torch

In [ ]:
world_size = 16
tensor_model_parallel_size = 2
pipeline_model_parallel_size = 4

In [ ]:
n_gpus_for_each_model = tensor_model_parallel_size * pipeline_model_parallel_size

In [ ]:
data_parallel_size = world_size // n_gpus_for_each_model

In [ ]:
data_parallel_size

2

In [ ]:
num_tensor_model_parallel_groups = world_size // tensor_model_paralell_size

In [ ]:
num_pipeline_model_parallel_groups = world_size // pipeline_model_parallel_size

In [ ]:
num_data_parallel_groups = world_size // data_parallel_size

In [ ]:
num_tensor_model_parallel_groups

8

Build data-parallel groups

In [ ]:
all_data_parallel_ranks = []

In [ ]:
for i in range()